# Individual Project
By Jeff Akins

#### ![EV Charging Station](https://i.insider.com/6042a8ac44d8e300117b9655?width=1200&format=jpeg)

## Exploring the State of the Electric Vehicle Charging Network within the U.S.
The world's methods of transportation are evolving, not only here in the United States, but globally. The EU recently proposed a mandate that all new vehicles be emissions free after 2035. In the U.S., the Biden administrations wants half of all new vehicles to be purely electric by 2030. Regardless of the mandates, buyers may be choosing to purchase EVs over their traditional internal combustion engine (ICE) counterparts in the upcoming years. No longer do you have to choose between the environment or the car that you want to drive. Many new EVs are surpassing comparable ICE vehicles in both speed, handling, and overall capability and new models of EVs are being introduced on a regular basis.

So, with all these new EV coming online, is the charging infrastructure keeping up? Buyers may be asking themselves, how many charging stations will be nearby if I need one? If I decide to go on a road trip, how difficult will it be to find an open charging station? I will try and answer these questions using a bit of machine learning to show the state of the charging networks now and the predicted growth over the next several years.

So, let's begin...

##### Imports:

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime
from sklearn.metrics import mean_squared_error
from math import sqrt

import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import register_matplotlib_converters

import statsmodels.api as sm
from statsmodels.tsa.api import Holt

import wrangle as w
import warnings
warnings.filterwarnings('ignore')

##### You can find the original dataset file [here](https://catalog.data.gov/dataset/alternative-fueling-station-locations-422f2/resource/341957d8-daf6-4a38-ab1d-8ec1bc21cfb9) on data.gov

In [2]:
csv_file_name = 'alternative-fuel-stations-2021-csv-3.csv'
ev = w.clean_ev(csv_file_name)

##### Data Cleaning:
The following steps were taken to clean the data:
- Dropped all rows that were not "electric" charging stations
- Dropped all columns with over 1000 null values
- Dropped an additional 12 columns that would not be used for this project
- Changed the pricing column to only include 'free' or 'cost' in the rows
- Filled the EV connector types nulls with the most common connector type of 'J1772'
- Fill EV Network nulls with most common 'non network' label
- Added values for 2 missing longitude values based on city location
- Renamed columns for ease of referencing and use with pandas
- Converted date columns to pandas datetime type
- Added a column for year that the charging station was opened
- Converted all 'count' columns into 'int' type and filled nulls with 0
- Replaced 'Public w/descriptor' with just 'Public' in group_access_code column
- Replaced 'Private w/descriptor' with just 'Private' in group_access_code column
- Dropped stations (rows) that are labeled as 'planned' and not yet accessible

Results are below:

In [3]:
ev.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47006 entries, 159 to 56793
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   fuel_code          47006 non-null  object        
 1   station_name       47006 non-null  object        
 2   address            47006 non-null  object        
 3   city               47006 non-null  object        
 4   state              47006 non-null  object        
 5   zip                47006 non-null  object        
 6   group_access_code  47006 non-null  object        
 7   level1_evse_num    47006 non-null  int64         
 8   level2_evse_num    47006 non-null  int64         
 9   ev_dc_fast_count   47006 non-null  int64         
 10  ev_network         47006 non-null  object        
 11  lat                47006 non-null  float64       
 12  long               47006 non-null  float64       
 13  update_date        47006 non-null  datetime64[ns]
 14  open

In [7]:
ev.head(2)

,fuel_code,station_name,address,city,state,zip,group_access_code,level1_evse_num,level2_evse_num,ev_dc_fast_count,ev_network,lat,long,update_date,open_date,connector_type,pricing,year_opened
159,ELEC,LADWP - Truesdale Center,11797 Truesdale St,Sun Valley,CA,91352,Private,0,39,3,Non-Networked,34.248319,-118.387971,2021-07-14,1999-10-15,CHADEMO J1772 J1772COMBO,Cost,1999
160,ELEC,LADWP - West LA District Office,1394 S Sepulveda Blvd,Los Angeles,CA,90024,Private,0,4,0,Non-Networked,34.052542,-118.448504,2021-07-14,2020-02-28,J1772,Free,2020
